# Importing Dataset

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import the drive module from google.colab
from google.colab import drive

# Mount the Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing Training Dataset

In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/drive/My Drive/SB ML Project Dataset 2/15MPa Data_Q2.xlsx'

# Read the sheets
df_train = pd.read_excel(file_path)  # First sheet

# We can now use combined_data for further processing or analysis
print(df_train.head())  # Display the first few rows of the combined dataframe
print(df_train.tail())

    t   P  G        T
0  60  15  1  7137.86
1  60  15  2  6838.09
2  60  15  3  6664.98
3  60  15  4  6552.86
4  60  15  5  6447.22
         t   P       G      T
367495  60  30  122496  300.0
367496  60  30  122497  300.0
367497  60  30  122498  300.0
367498  60  30  122499  300.0
367499  60  30  122500  300.0


In [ ]:
len(df_train)

367500

In [ ]:
# Check for null values in each column
null_counts = df_train.isnull().sum()
print("Null values in each column:\n", null_counts)

# Check the data types of each column
data_types = df_train.dtypes
print("\nData types of each column:\n", data_types)


Null values in each column:
 t    0
P    0
G    0
T    0
dtype: int64

Data types of each column:
 t      int64
P      int64
G      int64
T    float64
dtype: object


Importing test set

In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/drive/My Drive/SB ML Project Dataset 2/P=25 and 35 (2 sheet)_Test2.xlsx'

# Read the sheets
sheet1 = pd.read_excel(file_path, sheet_name=0)  # First sheet
sheet2 = pd.read_excel(file_path, sheet_name=1)  # Second sheet

# Concatenate the sheets
combined_data = pd.concat([sheet1, sheet2], ignore_index=True)

# We can now use combined_data for further processing or analysis
print(combined_data.head())  # Display the first few rows of the combined dataframe

   t=60   P  G        T
0    60  25  1  6245.50
1    60  25  2  6243.26
2    60  25  3  6241.65
3    60  25  4  6240.83
4    60  25  5  6239.70


In [ ]:
sheet1.head()

,t=60,P,G,T
0,60,25,1,6245.50
1,60,25,2,6243.26
2,60,25,3,6241.65
3,60,25,4,6240.83
4,60,25,5,6239.70


In [ ]:
sheet2.head()

,t=60,P,G,T
0,60,35,1,5923.48
1,60,35,2,5889.14
2,60,35,3,5855.25
3,60,35,4,5826.97
4,60,35,5,5812.39


In [ ]:
df_test=combined_data

In [ ]:
# Assuming df_test is your DataFrame
df_test = df_test.rename(columns={'t=60': 't'})

# Print the DataFrame to verify the change
print(df_test.head())

    t   P  G        T
0  60  25  1  6245.50
1  60  25  2  6243.26
2  60  25  3  6241.65
3  60  25  4  6240.83
4  60  25  5  6239.70


Scaling Train and Test Datasets

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
trainset_normalized = scaler.fit_transform(df_train)
testset_normalized =  scaler.fit_transform(df_test)

In [ ]:
print(trainset_normalized.shape)
print(testset_normalized.shape)

(367500, 4)
(245000, 4)


Preparing Sequences

In [ ]:
import numpy as np

def prepare_sequences(data, look_back):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :])
        y.append(data[i + look_back, -1])  # Assuming Temperature is the last column
    return np.array(X), np.array(y)

look_back = 12  # Best look-back window
X_train, y_train = prepare_sequences(trainset_normalized, look_back)
X_test, y_test = prepare_sequences(testset_normalized, look_back)

In [ ]:
X_train.shape

(367488, 12, 4)

In [ ]:
X_test.shape

(244988, 12, 4)

# Applying CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

input_shape = (look_back, X_train.shape[2])

In [ ]:
cnn_model = build_cnn_model(input_shape)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cnn_model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
9188/9188 [==============================] - 36s 3ms/step - loss: 0.0010 - val_loss: 1.1167e-05
Epoch 2/100
9188/9188 [==============================] - 31s 3ms/step - loss: 9.1881e-04 - val_loss: 1.2066e-05
Epoch 3/100
9188/9188 [==============================] - 31s 3ms/step - loss: 9.0599e-04 - val_loss: 3.5368e-06
Epoch 4/100
9188/9188 [==============================] - 31s 3ms/step - loss: 9.0008e-04 - val_loss: 3.6563e-05
Epoch 5/100
9188/9188 [==============================] - 31s 3ms/step - loss: 8.9833e-04 - val_loss: 1.0832e-05
Epoch 6/100
9188/9188 [==============================] - 31s 3ms/step - loss: 8.9495e-04 - val_loss: 4.0544e-06
Epoch 7/100
9188/9188 [==============================] - 31s 3ms/step - loss: 8.9064e-04 - val_loss: 5.8934e-06
Epoch 8/100
9188/9188 [==============================] - 31s 3ms/step - loss: 8.9041e-04 - val_loss: 5.5718e-06


In [ ]:
# Save the model to a file
cnn_model.save('cnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Make predictions
y_test_pred_cnn = cnn_model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Invert the predictions and actual values to compare with the original scale
predictions_original_test = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test_pred_cnn.reshape(-1, 1)), axis=1))[:, -1]
y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

# Calculate and print metrics
mse = mean_squared_error(y_test_original, predictions_original_test)
mae = mean_absolute_error(y_test_original, predictions_original_test)
mape = mean_absolute_percentage_error(y_test_original, predictions_original_test) * 100
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, predictions_original_test)

print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

7656/7656 [==============================] - 11s 1ms/step
MSE: 33664.9650344854
MAE: 21.137504779736275
MAPE: 2.8767148701818686
RMSE: 183.48014888397438
R2: 0.9748911427519761


In [ ]:
# Calculate evaluation metrics
from scipy.stats import pearsonr
correlation, _ = pearsonr(y_test_original, predictions_original_test)
print("Correlation Coefficient:", correlation)

Correlation Coefficient: 0.9873697607970351


In [ ]:
nrmse = rmse / np.mean(y_test_original)
print("Normalized RMSE (NRMSE):", nrmse)

Normalized RMSE (NRMSE): 0.2663674220402833


In [ ]:
nmae = mae / np.mean(y_test_original)
print("Normalized MAE (NMAE):", nmae)

Normalized MAE (NMAE): 0.030686385915802396


In [ ]:
min_abs_error = np.min(np.abs(y_test_original - predictions_original_test))
print("Minimum Absolute Error:", min_abs_error)

Minimum Absolute Error: 1.5559330563519325e-05


In [ ]:
max_abs_error = np.max(np.abs(y_test_original - predictions_original_test))
print("Maximum Absolute Error:", max_abs_error)

Maximum Absolute Error: 6021.356491345502


# Applying RNN

## LSTM: Look-back=12

In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


# Function to build the model with specific hyperparameters
def build_model_with_params(units, num_layers, units_last, input_shape):
    model = Sequential()
    # Add the first LSTM layer
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
    # Add intermediate LSTM layers
    for _ in range(num_layers):
        model.add(LSTM(units=units, return_sequences=True))
    # Add the last LSTM layer with return_sequences=False
    model.add(LSTM(units=units_last, return_sequences=False))
    # Add a Dense layer for final prediction
    model.add(Dense(units=1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Best Hyperparameters obtained from tuning
best_units = 40
best_num_layers = 2
best_units_last = 80

# Input shape based on your data
# input_shape = (X_train.shape[1], X_train.shape[2])

# Automate for different look-back windows
look_back_values = [12]

for look_back in look_back_values:
    #X, y = prepare_sequences(data_normalized, look_back)

    # Split the data into training, validation, and testing sets
    #X_train_current, X_temp, y_train_current, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    #X_val_current, X_test, y_val_current, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Prepare sequences for train set
    X_train_I, y_train_I = prepare_sequences(trainset_normalized, look_back)
    # Prepare sequences for test set
    X_test, y_test = prepare_sequences(testset_normalized, look_back)

    # Split the data into training, validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_I, y_train_I, test_size=0.2, random_state=42)

    # Build the model with specific hyperparameters
    current_model = build_model_with_params(units=best_units, num_layers=best_num_layers, units_last=best_units_last, input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model with EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    current_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Make predictions on the validation set
    y_test_pred = current_model.predict(X_test)

    # Invert the predictions and actual values to compare with the original scale
    predictions_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test_pred.reshape(-1, 1)), axis=1))[:, -1]
    y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

    # Calculate and print metrics
    mse = mean_squared_error(y_test_original, predictions_original)
    mae = mean_absolute_error(y_test_original, predictions_original)
    mape = mean_absolute_percentage_error(y_test_original, predictions_original)*100
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, predictions_original)

    print(f"Look Back: {look_back}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")
    print("--------------------")

Epoch 1/100
9188/9188 [==============================] - 84s 9ms/step - loss: 5.9649e-04 - val_loss: 3.7449e-04
Epoch 2/100
9188/9188 [==============================] - 77s 8ms/step - loss: 3.7466e-04 - val_loss: 3.7734e-04
Epoch 3/100
9188/9188 [==============================] - 79s 9ms/step - loss: 3.6752e-04 - val_loss: 3.4737e-04
Epoch 4/100
9188/9188 [==============================] - 78s 8ms/step - loss: 3.6383e-04 - val_loss: 3.6398e-04
Epoch 5/100
9188/9188 [==============================] - 77s 8ms/step - loss: 3.6150e-04 - val_loss: 3.5008e-04
Epoch 6/100
9188/9188 [==============================] - 76s 8ms/step - loss: 3.5936e-04 - val_loss: 3.5481e-04
Epoch 7/100
9188/9188 [==============================] - 77s 8ms/step - loss: 3.5787e-04 - val_loss: 3.4584e-04
Epoch 8/100
9188/9188 [==============================] - 76s 8ms/step - loss: 3.5721e-04 - val_loss: 3.5207e-04
Epoch 9/100
9188/9188 [==============================] - 76s 8ms/step - loss: 3.5662e-04 - val_loss: 3.4

In [ ]:
# Save the model to a file
current_model.save('lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Calculate evaluation metrics
from scipy.stats import pearsonr
correlation, _ = pearsonr(y_test_original, predictions_original)
print("Correlation Coefficient:", correlation)

Correlation Coefficient: 0.9938860221796841


In [ ]:
nrmse = rmse / np.mean(y_test_original)
print("Normalized RMSE (NRMSE):", nrmse)

Normalized RMSE (NRMSE): 0.18583048488742163


In [ ]:
nmae = mae / np.mean(y_test_original)
print("Normalized MAE (NMAE):", nmae)

Normalized MAE (NMAE): 0.018214255038949027


In [ ]:
min_abs_error = np.min(np.abs(y_test_original - predictions_original))
print("Minimum Absolute Error:", min_abs_error)

Minimum Absolute Error: 5.2118287442226574e-05


In [ ]:
max_abs_error = np.max(np.abs(y_test_original - predictions_original))
print("Maximum Absolute Error:", max_abs_error)

Maximum Absolute Error: 6028.713346786424


## LSTM: Look-back = 1

In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


# Function to build the model with specific hyperparameters
def build_model_with_params(units, num_layers, units_last, input_shape):
    model = Sequential()
    # Add the first LSTM layer
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
    # Add intermediate LSTM layers
    for _ in range(num_layers):
        model.add(LSTM(units=units, return_sequences=True))
    # Add the last LSTM layer with return_sequences=False
    model.add(LSTM(units=units_last, return_sequences=False))
    # Add a Dense layer for final prediction
    model.add(Dense(units=1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Best Hyperparameters obtained from tuning
best_units = 40
best_num_layers = 2
best_units_last = 80

# Input shape based on your data
# input_shape = (X_train.shape[1], X_train.shape[2])

# Automate for different look-back windows
look_back_values = [1]

for look_back in look_back_values:
    #X, y = prepare_sequences(data_normalized, look_back)

    # Split the data into training, validation, and testing sets
    #X_train_current, X_temp, y_train_current, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    #X_val_current, X_test, y_val_current, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Prepare sequences for train set
    X_train_I, y_train_I = prepare_sequences(trainset_normalized, look_back)
    # Prepare sequences for test set
    X_test, y_test = prepare_sequences(testset_normalized, look_back)

    # Split the data into training, validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_I, y_train_I, test_size=0.2, random_state=42)

    # Build the model with specific hyperparameters
    current_model = build_model_with_params(units=best_units, num_layers=best_num_layers, units_last=best_units_last, input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model with EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    current_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Make predictions on the validation set
    y_test_pred = current_model.predict(X_test)

    # Invert the predictions and actual values to compare with the original scale
    predictions_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test_pred.reshape(-1, 1)), axis=1))[:, -1]
    y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

    # Calculate and print metrics
    mse = mean_squared_error(y_test_original, predictions_original)
    mae = mean_absolute_error(y_test_original, predictions_original)
    mape = mean_absolute_percentage_error(y_test_original, predictions_original)*100
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, predictions_original)

    print(f"Look Back: {look_back}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")
    print("--------------------")

Epoch 1/100
9188/9188 [==============================] - 82s 8ms/step - loss: 6.5091e-04 - val_loss: 4.1015e-04
Epoch 2/100
9188/9188 [==============================] - 72s 8ms/step - loss: 3.9055e-04 - val_loss: 3.7400e-04
Epoch 3/100
9188/9188 [==============================] - 72s 8ms/step - loss: 3.8636e-04 - val_loss: 3.7456e-04
Epoch 4/100
9188/9188 [==============================] - 72s 8ms/step - loss: 3.8162e-04 - val_loss: 3.7834e-04
Epoch 5/100
9188/9188 [==============================] - 72s 8ms/step - loss: 3.7910e-04 - val_loss: 3.8353e-04
Epoch 6/100
9188/9188 [==============================] - 72s 8ms/step - loss: 3.7742e-04 - val_loss: 3.7015e-04
Epoch 7/100
9188/9188 [==============================] - 72s 8ms/step - loss: 3.7565e-04 - val_loss: 4.0954e-04
Epoch 8/100
9188/9188 [==============================] - 82s 9ms/step - loss: 3.7494e-04 - val_loss: 4.4711e-04
Epoch 9/100
9188/9188 [==============================] - 76s 8ms/step - loss: 3.7414e-04 - val_loss: 3.7

In [ ]:
# Calculate evaluation metrics
from scipy.stats import pearsonr
correlation, _ = pearsonr(y_test_original, predictions_original)
print("Correlation Coefficient:", correlation)

Correlation Coefficient: 0.9936122100985549


In [ ]:
nrmse = rmse / np.mean(y_test_original)
print("Normalized RMSE (NRMSE):", nrmse)

Normalized RMSE (NRMSE): 0.19001194998189228


In [ ]:
nmae = mae / np.mean(y_test_original)
print("Normalized MAE (NMAE):", nmae)

Normalized MAE (NMAE): 0.028144110031770454


In [ ]:
min_abs_error = np.min(np.abs(y_test_original - predictions_original))
print("Minimum Absolute Error:", min_abs_error)

Minimum Absolute Error: 0.0001863662524783649


In [ ]:
max_abs_error = np.max(np.abs(y_test_original - predictions_original))
print("Maximum Absolute Error:", max_abs_error)

Maximum Absolute Error: 6047.020416950974


## LSTM: Look-back=24

In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


# Function to build the model with specific hyperparameters
def build_model_with_params(units, num_layers, units_last, input_shape):
    model = Sequential()
    # Add the first LSTM layer
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
    # Add intermediate LSTM layers
    for _ in range(num_layers):
        model.add(LSTM(units=units, return_sequences=True))
    # Add the last LSTM layer with return_sequences=False
    model.add(LSTM(units=units_last, return_sequences=False))
    # Add a Dense layer for final prediction
    model.add(Dense(units=1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Best Hyperparameters obtained from tuning
best_units = 40
best_num_layers = 2
best_units_last = 80

# Input shape based on your data
# input_shape = (X_train.shape[1], X_train.shape[2])

# Automate for different look-back windows
look_back_values = [24]

for look_back in look_back_values:
    #X, y = prepare_sequences(data_normalized, look_back)

    # Split the data into training, validation, and testing sets
    #X_train_current, X_temp, y_train_current, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    #X_val_current, X_test, y_val_current, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Prepare sequences for train set
    X_train_I, y_train_I = prepare_sequences(trainset_normalized, look_back)
    # Prepare sequences for test set
    X_test, y_test = prepare_sequences(testset_normalized, look_back)

    # Split the data into training, validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_I, y_train_I, test_size=0.2, random_state=42)

    # Build the model with specific hyperparameters
    current_model = build_model_with_params(units=best_units, num_layers=best_num_layers, units_last=best_units_last, input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model with EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    current_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Make predictions on the validation set
    y_test_pred = current_model.predict(X_test)

    # Invert the predictions and actual values to compare with the original scale
    predictions_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test_pred.reshape(-1, 1)), axis=1))[:, -1]
    y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

    # Calculate and print metrics
    mse = mean_squared_error(y_test_original, predictions_original)
    mae = mean_absolute_error(y_test_original, predictions_original)
    mape = mean_absolute_percentage_error(y_test_original, predictions_original)*100
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, predictions_original)

    print(f"Look Back: {look_back}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")
    print("--------------------")

Epoch 1/100
9187/9187 [==============================] - 101s 10ms/step - loss: 6.2635e-04 - val_loss: 3.8886e-04
Epoch 2/100
9187/9187 [==============================] - 95s 10ms/step - loss: 3.6675e-04 - val_loss: 3.8470e-04
Epoch 3/100
9187/9187 [==============================] - 92s 10ms/step - loss: 3.5871e-04 - val_loss: 3.9533e-04
Epoch 4/100
9187/9187 [==============================] - 93s 10ms/step - loss: 3.5621e-04 - val_loss: 3.8082e-04
Epoch 5/100
9187/9187 [==============================] - 93s 10ms/step - loss: 3.5465e-04 - val_loss: 3.9119e-04
Epoch 6/100
9187/9187 [==============================] - 93s 10ms/step - loss: 3.5268e-04 - val_loss: 3.8737e-04
Epoch 7/100
9187/9187 [==============================] - 93s 10ms/step - loss: 3.5194e-04 - val_loss: 3.8049e-04
Epoch 8/100
9187/9187 [==============================] - 93s 10ms/step - loss: 3.5072e-04 - val_loss: 4.6405e-04
Epoch 9/100
9187/9187 [==============================] - 93s 10ms/step - loss: 3.5016e-04 - val

In [ ]:
# Calculate evaluation metrics
from scipy.stats import pearsonr
correlation, _ = pearsonr(y_test_original, predictions_original)
print("Correlation Coefficient:", correlation)

Correlation Coefficient: 0.9938735761619273


In [ ]:
nrmse = rmse / np.mean(y_test_original)
print("Normalized RMSE (NRMSE):", nrmse)

Normalized RMSE (NRMSE): 0.18642063720837176


In [ ]:
nmae = mae / np.mean(y_test_original)
print("Normalized MAE (NMAE):", nmae)

Normalized MAE (NMAE): 0.01797094192455359


In [ ]:
min_abs_error = np.min(np.abs(y_test_original - predictions_original))
print("Minimum Absolute Error:", min_abs_error)

Minimum Absolute Error: 7.753073987259995e-07


In [ ]:
max_abs_error = np.max(np.abs(y_test_original - predictions_original))
print("Maximum Absolute Error:", max_abs_error)

Maximum Absolute Error: 6033.861546027297


## GRU

In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


# Function to build the model with specific hyperparameters
def build_model_with_params(units, num_layers, units_last, input_shape):
    model = Sequential()
    # Add the first GRU layer
    model.add(GRU(units=units, return_sequences=True, input_shape=input_shape))
    # Add intermediate GRU layers
    for _ in range(num_layers):
        model.add(GRU(units=units, return_sequences=True))
    # Add the last GRU layer with return_sequences=False
    model.add(GRU(units=units_last, return_sequences=False))
    # Add a Dense layer for final prediction
    model.add(Dense(units=1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Best Hyperparameters obtained from tuning
best_units = 50
best_num_layers = 4
best_units_last = 30

# Input shape based on your data
# input_shape = (X_train.shape[1], X_train.shape[2])

# Automate for different look-back windows
look_back_values = [2]

for look_back in look_back_values:

    # Prepare sequences for train set
    X_train_I, y_train_I = prepare_sequences(trainset_normalized, look_back)
    # Prepare sequences for test set
    X_test, y_test = prepare_sequences(testset_normalized, look_back)

    # Split the data into training, validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_I, y_train_I, test_size=0.2, random_state=42)

    # Build the model with specific hyperparameters
    current_model = build_model_with_params(units=best_units, num_layers=best_num_layers, units_last=best_units_last, input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model with EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    current_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Make predictions on the validation set
    y_test_pred = current_model.predict(X_test)

    # Invert the predictions and actual values to compare with the original scale
    predictions_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test_pred.reshape(-1, 1)), axis=1))[:, -1]
    y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

    # Calculate and print metrics
    mse = mean_squared_error(y_test_original, predictions_original)
    mae = mean_absolute_error(y_test_original, predictions_original)
    mape = mean_absolute_percentage_error(y_test_original, predictions_original)*100
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, predictions_original)

    print(f"Look Back: {look_back}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")
    print("--------------------")

Epoch 1/100
9188/9188 [==============================] - 94s 9ms/step - loss: 5.1744e-04 - val_loss: 3.4704e-04
Epoch 2/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.9751e-04 - val_loss: 3.5072e-04
Epoch 3/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.8629e-04 - val_loss: 3.3166e-04
Epoch 4/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.8090e-04 - val_loss: 4.5620e-04
Epoch 5/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.7626e-04 - val_loss: 3.3098e-04
Epoch 6/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.7322e-04 - val_loss: 3.2953e-04
Epoch 7/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.7062e-04 - val_loss: 3.2893e-04
Epoch 8/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.6914e-04 - val_loss: 3.4262e-04
Epoch 9/100
9188/9188 [==============================] - 84s 9ms/step - loss: 3.6766e-04 - val_loss: 3.4

In [ ]:
# Save the model to a file
current_model.save('gru_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Calculate evaluation metrics
from scipy.stats import pearsonr
correlation, _ = pearsonr(y_test_original, predictions_original)
print("Correlation Coefficient:", correlation)

Correlation Coefficient: 0.993843982699964


In [ ]:
nrmse = rmse / np.mean(y_test_original)
print("Normalized RMSE (NRMSE):", nrmse)

Normalized RMSE (NRMSE): 0.18659806840422208


In [ ]:
nmae = mae / np.mean(y_test_original)
print("Normalized MAE (NMAE):", nmae)

Normalized MAE (NMAE): 0.017300679871673826


In [ ]:
min_abs_error = np.min(np.abs(y_test_original - predictions_original))
print("Minimum Absolute Error:", min_abs_error)

Minimum Absolute Error: 1.9321142758599308e-05


In [ ]:
max_abs_error = np.max(np.abs(y_test_original - predictions_original))
print("Maximum Absolute Error:", max_abs_error)

Maximum Absolute Error: 6040.879788501597


# Remarks

RNN models, including LSTM and GRU, are typically designed to handle sequential data where the temporal order and dependencies between the data points are crucial. They are particularly effective in tasks such as time series forecasting, natural language processing, and other applications where the context from previous time steps significantly influences the prediction of future time steps.

Considering the dataset:
Constant Time (t):
The t (time) column is constant at 60. This indicates that there is no temporal progression or sequence in the time domain, which is one of the fundamental aspects that RNNs, LSTMs, and GRUs are designed to handle.

Pressure (P) and Grid (G):
The P (Pressure) and G (Grid) columns vary, but these variations do not represent a sequential relationship over time. Instead, they appear to be independent variables that might influence the target variable T (Temperature).

Temperature (T):
The T (Temperature) values are continuous and is the target variable.

Applicability of RNN Models:
Given that RNN models are designed to capture temporal dependencies and the provided dataset does not have a sequential time component (as time t is constant), it does not make sense to apply RNN models in this context. Instead, the dataset appears to be more suited to regression models that can handle multiple input features (Pressure, Grid) to predict the target variable (Temperature).

Suitable Models Include:
Linear Regression:

A simple linear regression model could be effective if the relationship between P, G, and T is linear.

Decision Trees and Random Forests:
These models can handle non-linear relationships and interactions between features effectively.

Gradient Boosting Machines (GBM):
GBMs, including XGBoost and LightGBM, can capture complex patterns and interactions in the data.

Neural Networks:
A feedforward neural network (FNN) or a multilayer perceptron (MLP) could also be appropriate for capturing non-linear relationships in the data.

Conclusion:
Applying RNN models to a dataset without a meaningful temporal sequence might not be effective. Given the structure of the dataset, it would be more appropriate to use regression models or non-sequential neural networks to predict the target variable T (Temperature) based on the input features P (Pressure) and G (Grid).